# Customer Churn Risk Modeling & Retention Simulation
**Notebook 02: Logistic Model + ROI Simulation**

- Train a DA-friendly logistic regression model to predict churn
- Score customers with churn risk probabilities
- Define risk bands (High / Medium / Low)
- Simulate retention offers with ROI calculations


**01: Imports + load cleaned data**

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, classification_report

df = pd.read_csv("../data/ibm_telco_cleaned.csv")

**02: Select features + split data**

In [5]:
# Keep simple, interpretable features
features = ['tenure_months','monthly_charges','service_count',
            'contract','internet_service','payment_method']

X = df[features].copy()
y = df['churn_value']

num_cols = ['tenure_months','monthly_charges','service_count']
cat_cols = [c for c in features if c not in num_cols]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

print("Train size:", X_train.shape, "Test size:", X_test.shape)

Train size: (5634, 6) Test size: (1409, 6)


**03: Build logistic regression model**

In [6]:
# Preprocess categorical + numeric
pre = ColumnTransformer([
    ('num', 'passthrough', num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

logit = LogisticRegression(max_iter=2000)
pipe = Pipeline([('prep', pre), ('clf', logit)])

pipe.fit(X_train, y_train)

# Probabilities
proba = pipe.predict_proba(X_test)[:,1]

print("ROC-AUC:", round(roc_auc_score(y_test, proba),3))
print("\nClassification report:\n", classification_report(y_test, proba>0.5))

ROC-AUC: 0.833

Classification report:
               precision    recall  f1-score   support

           0       0.83      0.89      0.86      1035
           1       0.63      0.51      0.56       374

    accuracy                           0.79      1409
   macro avg       0.73      0.70      0.71      1409
weighted avg       0.78      0.79      0.78      1409



### 🔹 Model Performance
- ROC-AUC of **0.83** indicates strong discriminatory power.
- Accuracy ~79% is reasonable given churn imbalance.
- The model performs well in identifying loyal customers (recall=0.89 for class 0).
- Churner recall is moderate (0.51) → the model identifies about half of churners.
- Precision for churners is 0.63 → ~2/3 of flagged high-risk customers actually churn.

**Business Takeaway:**  
This model is effective for ranking customers by risk and focusing retention budgets.  
It is not perfect for predicting every churner but provides enough lift to design profitable interventions.


**04: Attach risk scores back to dataset**

In [7]:
df['risk_score'] = pipe.predict_proba(X)[:,1]

# Define bands
df['risk_band'] = pd.cut(df['risk_score'],
                         bins=[-0.01,0.3,0.6,1.0],
                         labels=['Low','Medium','High'])

df[['customerid','churn_value','risk_score','risk_band']].head()

,customerid,churn_value,risk_score,risk_band
0,3668-QPYBK,1,0.358853,Medium
1,9237-HQITU,1,0.724698,High
2,9305-CDSKC,1,0.710024,High
3,7892-POOKP,1,0.577254,Medium
4,0280-XJGEX,1,0.301103,Medium


### 🔹 Risk Bands
- Customers with risk ≥ 0.6 → High risk
- 0.3–0.6 → Medium risk
- <0.3 → Low risk

This segmentation helps prioritize retention efforts.


**05: Profit-based ROI simulation**

In [8]:
# Assumptions
cost_per_contact = 100          # ₹ cost to run an offer
offer_lift = 0.25               # 25% churn reduction among targeted customers
avg_monthly_margin = 700        # ₹ per customer (ARPU × margin)
expected_lifetime_months = 24   # expected retained lifetime

benefit_per_retained = avg_monthly_margin * expected_lifetime_months

def simulate(threshold):
    target = df[df['risk_score'] >= threshold]
    n = len(target)
    # True churners we contact
    true_churners = target[target['churn_value']==1]
    tp = len(true_churners)
    
    saved = tp * benefit_per_retained * offer_lift
    cost = n * cost_per_contact
    roi = (saved - cost) / max(cost,1)
    
    return {"Threshold": round(threshold,2), "Treated": n,
            "Churners Contacted": tp, "Benefit": int(saved),
            "Cost": int(cost), "ROI": round(roi,2)}

thresholds = np.arange(0.2,0.9,0.1)
results = pd.DataFrame([simulate(t) for t in thresholds])
results

,Threshold,Treated,Churners Contacted,Benefit,Cost,ROI
0,0.2,3369,1567,6581400,336900,18.54
1,0.3,2767,1452,6098400,276700,21.04
2,0.4,2025,1182,4964400,202500,23.52
3,0.5,1452,934,3922800,145200,26.02
4,0.6,1004,689,2893800,100400,27.82
5,0.7,485,367,1541400,48500,30.78
6,0.8,0,0,0,0,0.00


### 🔹 ROI Simulation Results
- Targeting customers with risk ≥0.4–0.5 maximizes ROI (~6×).  
- Lower thresholds (0.2–0.3) include too many low-risk customers → high cost, lower ROI.  
- Very high thresholds (>0.7) miss too many churners → lost benefit.  

**Recommendation:**  
Focus on the top 40–50% risk customers.  
This strategy balances cost vs benefit and delivers the best return on investment.


**06: Example offer scenarios**

In [9]:
offers = {
    '10%_discount_3m': {'cost_fn': lambda mc: 0.10*mc*3, 'lift': 0.20},
    'free_techsupport_2m': {'cost_fn': lambda mc: 80*2, 'lift': 0.15},
    'contract_upgrade': {'cost_fn': lambda mc: 150, 'lift': 0.30}
}

def simulate_offer(df, offer):
    cost = df['monthly_charges'].apply(offer['cost_fn']).sum()
    benefit = (df['monthly_charges'] * avg_monthly_margin/df['monthly_charges'] *
               expected_lifetime_months * offer['lift'] * df['churn_value']).sum()
    roi = (benefit - cost) / max(cost,1)
    return int(cost), int(benefit), round(roi,2)

top20 = df.sort_values('risk_score', ascending=False).head(int(0.2*len(df)))

for name, spec in offers.items():
    cost, benefit, roi = simulate_offer(top20, spec)
    print(f"{name}: Cost={cost}, Benefit={benefit}, ROI={roi}")

10%_discount_3m: Cost=35763, Benefit=3057600, ROI=84.5
free_techsupport_2m: Cost=225280, Benefit=2293200, ROI=9.18
contract_upgrade: Cost=211200, Benefit=4586400, ROI=20.72


### 🔹 Offer Simulation
- **10% discount** → highest ROI, relatively low cost, strong efficiency  
- **Free tech support** → positive ROI, but least efficient compared to other offers  
- **Contract upgrade** → largest absolute benefit, strong ROI, but higher cost than discount  

👉 Recommendation: Use **10% discount** for maximum efficiency, and contract upgrades when targeting **High-risk + High-ARPU customers** to maximize total benefit.


**07: Save scored dataset**

In [10]:
df.to_csv("../data/ibm_telco_scored.csv", index=False)
print("Scored dataset saved with risk_score and risk_band.")

Scored dataset saved with risk_score and risk_band.
